# get data from CSV and TSV (Flat File)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/bestofrt.tsv' ,sep='\t')

In [3]:
df.head()

,ranking,critic_score,title,number_of_critic_ratings
0,1,99,The Wizard of Oz (1939),110
1,2,100,Citizen Kane (1941),75
2,3,100,The Third Man (1949),77
3,4,99,Get Out (2017),282
4,5,97,Mad Max: Fury Road (2015),370


# NOTE

# get data from web using Web Scrapng

# FIRST EXAMPLE

In [4]:
import requests
from bs4 import BeautifulSoup
import re

In [5]:
url = 'https://news.ycombinator.com/news'
request = requests.get(url)

In [6]:
print(request.status_code ,request.reason)

200 OK


In [7]:
html_content = request.content
html_soup = BeautifulSoup(html_content,'html.parser')
a=[]

In [8]:
for item in html_soup.find_all('tr' ,class_='athing'):
    item_a = item.find('a',class_='storylink')
    item_link = item_a.get('href') if item_a else None
    item_text = item_a.get_text(strip =True) if item_a else None
    item_web = item.find(class_='sitestr')
    item_we = item_web.get_text(strip=True) if item_web else None
    next_row = item.find_next_sibling('tr')
    item_score = next_row.find('span',class_='score')
    item_sc=item_score.get_text(strip=True)if item_score else '0'
    item_ag = next_row.find('span',class_='age')
    item_age =item_ag.get_text(strip=True) if item_ag else None
    item_comments =next_row.find('a', string=re.compile('\d+(&nbsp;|\s)comment(s?)'))
    item_comments = item_comments.get_text(strip=True) if item_comments else None
    a.append({'item_link':item_link,
             'item_text':item_text,
             'item_web':item_web,
             'item_sc':item_sc,
             'item_age':item_age,
             'item_comments':item_comments})

In [9]:
df_2 = pd.DataFrame(a,columns={'item_link':item_link,'item_text':item_text,'item_web':item_web,'item_sc':item_sc,'item_age':item_age,'item_comments':item_comments})

In [10]:
df_2.head()

,item_link,item_text,item_web,item_sc,item_age,item_comments
0,https://blogs.sciencemag.org/pipeline/archives...,Sand won't save you this time (2008),"<span class=""sitestr"">sciencemag.org</span>",54 points,2 hours ago,7 comments
1,https://fedoramagazine.org/what-is-silverblue/,What is Silverblue?,"<span class=""sitestr"">fedoramagazine.org</span>",176 points,6 hours ago,63 comments
2,https://sydney.edu.au/news-opinion/news/2019/0...,"Insects feel persistent pain after injury, evi...","<span class=""sitestr"">sydney.edu.au</span>",106 points,5 hours ago,60 comments
3,https://www.polygon.com/2019/4/2/18286977/eve-...,One player spent 10 years exploring every corn...,"<span class=""sitestr"">polygon.com</span>",257 points,10 hours ago,185 comments
4,http://www.petesqbsite.com/index.php,Pete's QBASIC / QuickBasic Site,"<span class=""sitestr"">petesqbsite.com</span>",83 points,6 hours ago,41 comments


# SECONDE EXAMPLE

In [11]:
from urllib.parse import urljoin ,urlparse

In [12]:
book_req= requests.get('http://quotes.toscrape.com/')
print(book_req.status_code , book_req.reason)

200 OK


In [13]:
quote_content = BeautifulSoup(book_req.content , 'html.parser')

In [14]:
base_url ='http://quotes.toscrape.com/'
def clean_URL(auther_url):
    url = urljoin(base_url , auther_url)
    return url
clean_URL('author/Albert-Einstein/')

'http://quotes.toscrape.com/author/Albert-Einstein/'

In [15]:
author_url = []
quote_list=[]
AUTHOR=[]
def scrap_Quotes(quote_content):
    for quote in quote_content.find_all('div',class_='quote'):
        quote_text = quote.find(class_='text').get_text(strip=True)
        quote_author = quote.find(class_='author').get_text(strip=True)
        author_link = quote.find('a',string='(about)').get('href')
        author_url.append(author_link)
        quote_list.append({
            'quote_text':quote_text,
            'quote_author':quote_author,
            'author_link':author_link
        })
def Scrap_Author(author_content):
    author_name = author_content.find('h3',class_='author-title').get_text(strip=True)
    author_date = author_content.find('span',class_='author-born-date').get_text(strip=True)
    author_loc = author_content.find('span',class_='author-born-location').get_text(strip=True)
    author_desc = author_content.find('div',class_='author-description')
    auth_desc = author_desc.get_text(strip=True) if author_desc else None
    AUTHOR.append({
        'author_name':author_name,
        'author_date':author_date,
        'author_loc':author_loc,
        'author_desc':author_desc,
        'auth_desc':auth_desc
    })

In [16]:
#main
url =base_url
while True:
    print('start scarping ..... ', url)
    book_req= requests.get(url)
    quote_content = BeautifulSoup(book_req.content , 'html.parser')    
    scrap_Quotes(quote_content)
    next_a = quote_content.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break
    url = urljoin(url, next_a[0].get('href'))
    for auth in author_url:
        aut_URL = clean_URL(auth)
        print('Start Scraping AUTH ......', aut_URL)
        auth_req = requests.get(aut_URL)
        auth_cont = BeautifulSoup(auth_req.content,'html.parser')
        Scrap_Author(auth_cont)

start scarping .....  http://quotes.toscrape.com/
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Albert-Einstein
Start Scraping AUTH ...... http://quotes.toscrape.com/author/J-K-Rowling
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Albert-Einstein
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Jane-Austen
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Marilyn-Monroe
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Albert-Einstein
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Andre-Gide
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Thomas-A-Edison
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Eleanor-Roosevelt
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Steve-Martin
start scarping .....  http://quotes.toscrape.com/page/2/
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Albert-Einstein
Start Scraping AUTH ...... http://quotes.toscrape.com/author/

Start Scraping AUTH ...... http://quotes.toscrape.com/author/Andre-Gide
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Thomas-A-Edison
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Eleanor-Roosevelt
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Steve-Martin
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Marilyn-Monroe
Start Scraping AUTH ...... http://quotes.toscrape.com/author/J-K-Rowling
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Albert-Einstein
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Bob-Marley
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Dr-Seuss
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Douglas-Adams
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Elie-Wiesel
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Friedrich-Nietzsche
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Mark-Twain
Start Scraping AUTH ...... ht

Start Scraping AUTH ...... http://quotes.toscrape.com/author/Marilyn-Monroe
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Albert-Einstein
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Andre-Gide
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Thomas-A-Edison
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Eleanor-Roosevelt
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Steve-Martin
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Marilyn-Monroe
Start Scraping AUTH ...... http://quotes.toscrape.com/author/J-K-Rowling
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Albert-Einstein
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Bob-Marley
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Dr-Seuss
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Douglas-Adams
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Elie-Wiesel
Start Scraping AUTH ...... ht

Start Scraping AUTH ...... http://quotes.toscrape.com/author/Marilyn-Monroe
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Marilyn-Monroe
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Albert-Einstein
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Marilyn-Monroe
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Marilyn-Monroe
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Martin-Luther-King-Jr
Start Scraping AUTH ...... http://quotes.toscrape.com/author/J-K-Rowling
Start Scraping AUTH ...... http://quotes.toscrape.com/author/James-Baldwin
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Jane-Austen
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Eleanor-Roosevelt
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Marilyn-Monroe
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Albert-Einstein
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Haruki-Murakami
Start 

Start Scraping AUTH ...... http://quotes.toscrape.com/author/Alfred-Tennyson
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Charles-Bukowski
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Terry-Pratchett
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Dr-Seuss
Start Scraping AUTH ...... http://quotes.toscrape.com/author/J-D-Salinger
Start Scraping AUTH ...... http://quotes.toscrape.com/author/George-Carlin
Start Scraping AUTH ...... http://quotes.toscrape.com/author/John-Lennon
Start Scraping AUTH ...... http://quotes.toscrape.com/author/W-C-Fields
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Ayn-Rand
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Mark-Twain
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Albert-Einstein
Start Scraping AUTH ...... http://quotes.toscrape.com/author/Jane-Austen
Start Scraping AUTH ...... http://quotes.toscrape.com/author/J-K-Rowling
Start Scraping AUTH ...... http://quote

In [17]:
quote_list

[{'quote_text': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
  'quote_author': 'Albert Einstein',
  'author_link': '/author/Albert-Einstein'},
 {'quote_text': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
  'quote_author': 'J.K. Rowling',
  'author_link': '/author/J-K-Rowling'},
 {'quote_text': '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
  'quote_author': 'Albert Einstein',
  'author_link': '/author/Albert-Einstein'},
 {'quote_text': '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
  'quote_author': 'Jane Austen',
  'author_link': '/author/Jane-Austen'},
 {'quote_text': "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
  'quote_author': 'Marilyn Monroe',
  'au

In [18]:
Quote_df =pd.DataFrame(quote_list)

In [19]:
Quote_df.head()

,author_link,quote_author,quote_text
0,/author/Albert-Einstein,Albert Einstein,“The world as we have created it is a process ...
1,/author/J-K-Rowling,J.K. Rowling,"“It is our choices, Harry, that show what we t..."
2,/author/Albert-Einstein,Albert Einstein,“There are only two ways to live your life. On...
3,/author/Jane-Austen,Jane Austen,"“The person, be it gentleman or lady, who has ..."
4,/author/Marilyn-Monroe,Marilyn Monroe,"“Imperfection is beauty, madness is genius and..."


# THIRD EXAMPLE 

# get text from web as .txt File


In [24]:
review_url =['https://2.python-requests.org//en/latest/user/quickstart/#binary-response-content.txt']
import os 
folder_name ='text_review'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
for url in review_url:
    resp =requests.get(url)
    with open(os.path.join(folder_name , url.split('/')[-1]),mode='wb') as file:
        file.write(resp.content)

# get data from API
